In [ ]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import pandas as pd
import math
import operator
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

In [ ]:
import os
os.environ["OMP_NUM_THREADS"] = '1'

In [ ]:
from sklearn.datasets import load_iris
iris = load_iris(as_frame=True)
x_label='sepal length (cm)'
y_label='petal length (cm)'
cols = list(iris.frame.columns)[:-1]
df=iris.frame

In [ ]:
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
X = StandardScaler().fit_transform(df.iloc[:,0:4])
pd.DataFrame(X).describe()

In [ ]:
import holoviews as hv
from holoviews import opts
hv.extension('bokeh')
import panel as pn
import hvplot.pandas

In [ ]:
x = pn.widgets.Select(name='x', options=cols, value = x_label)
y = pn.widgets.Select(name='y', options=cols, value = y_label)
check = pn.widgets.Checkbox (name='Классы', value = True)

color1=['blue', 'orange', 'green']
color2=['red', 'red', 'red']

def draw(x, y, check):
    class_plot=df.hvplot.scatter(x, y, by='target', marker='o', size=100,  color=( color1 if check else color2) )
    centers = df.groupby('target')[[x] if x == y else [x, y]].mean()
    centers_plot= hv.Scatter(centers,  label='Class centers')
    centers_plot.opts(color='k', marker='x', size=15, line_width=5)
    fig = (class_plot * centers_plot) if check else class_plot
    fig.opts(xlabel=x, ylabel=y, width=600, height=500, tools=['hover'], 
             title="Iris Dataset", legend_position='top_left', show_grid=True)
    return ( fig  )
    
pn.Row(
    pn.pane.HoloViews(
        pn.bind(draw, x, y, check)
    ).servable(),
    
    pn.WidgetBox(
        pn.Column(
           "Обучающая выборка",
            check, 
            x, y
        ).servable(target='sidebar')
    ),
)


In [ ]:
from sklearn.cluster import KMeans
from yellowbrick.cluster import KElbowVisualizer
# Instantiate a scikit-learn K-Means model
model = KMeans(random_state=0, n_init='auto' )
# Instantiate the KElbowVisualizer with the number of clusters and the metric 
visualizer = KElbowVisualizer(model, k=(1,12), timings=False)
# Fit the data and visualize
visualizer.fit(X)   
visualizer.poof() 

In [ ]:
visualizer = KElbowVisualizer(model, k=(2,12), metric='silhouette', timings=False)
# Fit the data and visualize
visualizer.fit(X)   
visualizer.poof() 

In [ ]:
import import_ipynb
from pyclustertend import  vat,  ivat, hopkins

print(hopkins(X, X.shape[0]))
vat(X, figure_size=(5, 3))
ivat(X, figure_size=(7, 5))

In [ ]:
x = pn.widgets.Select(name='x', options=cols, value = x_label)
y = pn.widgets.Select(name='y', options=cols, value = y_label)
n_clusters = pn.widgets.IntSlider(name='n_clusters', start=1, end=12, value=3)
check = pn.widgets.Checkbox (name='Исходные классы', value = False)

def get_clusters(x, y, n_clusters, check):
    kmeans = KMeans(n_clusters=n_clusters, n_init=10, random_state = 123, verbose = 0)
    kmeans.fit(X)
    pred=kmeans.fit_predict(X)
    
    df['cluster'] = pred
    cluster_centers = df.groupby('cluster')[[x] if x == y else [x, y]].mean()
    
    cluster_plot=df.hvplot.scatter(x, y, by='cluster', marker='o', size=100,  color=['orange', 'blue', 'green', 'cyan', 'magenta', 'yellow', 'red'])
    
    class_plot=df.hvplot.scatter(x, y, by='target', marker='o', size=50,  color=['blue', 'orange', 'green'])
    
    cluster_centers_plot= hv.Scatter(cluster_centers,  label='Cluster centers')
    cluster_centers_plot.opts(color='k', marker='x', size=15, line_width=5)
    
    centers = df.groupby('target')[[x] if x == y else [x, y]].mean()
    centers_plot= hv.Scatter(centers,  label='Class centers')
    centers_plot.opts( marker='x', size=15, line_width=5, color='red')
    
    #*  centers_plot
    fig =  (cluster_plot  * class_plot * cluster_centers_plot * centers_plot) if check else (cluster_plot  * cluster_centers_plot)
    fig.opts(xlabel=x, ylabel=y, width=600, height=500, tools=['hover'], 
             title="Iris K-Means++ Clustering", legend_position='top_left', show_grid=True)
    return ( fig  )
    

pn.Row(
    pn.pane.HoloViews(
        pn.bind(get_clusters, x, y, n_clusters, check)
    ).servable(),
    
    pn.WidgetBox(
        pn.Column(
            "Признаки",
            x, y, check, 
            "Кластеризация KMeans++",
            n_clusters
        ).servable(target='sidebar')
    ),
)

Оценка качества кластеризации

In [ ]:
from sklearn import metrics
import time
max_clusters=6

metrics_name = [ 'homogeneity_score', 'completeness_score', 'rand_score']
    
clustering_metrics = [
       
        metrics.homogeneity_score,
        metrics.completeness_score,
        metrics.rand_score   
    ]
results = pd.DataFrame(columns=['n_clusters', 'homogeneity_score', 'completeness_score', 'rand_score'])
np.random.seed(1234)

for i in range(1,max_clusters+1):
    res=[]
    print("%i clusters" % (i) ) 
    t0 = time.time()
    kmeans = KMeans(n_clusters=i, n_init=10)
    kmeans.fit(X)
    clusters = kmeans.fit_predict(X)
    df['cluster'] = kmeans.fit_predict(X)
    
    res.append(i)
    for name, m in zip(metrics_name, clustering_metrics):
        res.append(m(df['target'],df['cluster']))
    t1 = time.time()
    dt=(t1 - t0)
    print ('Time = %.3f sec.' % (dt) )
    results.loc[len(results)] = res
results   

In [ ]:
results.hvplot.line( x='n_clusters',   
                  line_width=3,  xlim=(1, max_clusters),
                  legend='top',   widget_location='left',
                  grid = True, xlabel='Число кластеров', ylabel='Значение',
                  height=400, width=700)

In [ ]:
from sklearn.cluster import DBSCAN
db = DBSCAN(eps=0.75, min_samples=15).fit(X)
labels = db.labels_

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)

print("Estimated number of clusters: %d" % n_clusters_)
print("Estimated number of noise points: %d" % n_noise_)

In [ ]:
x = pn.widgets.Select(name='x', options=cols, value = x_label)
y = pn.widgets.Select(name='y', options=cols, value = y_label)
epsilon = pn.widgets.FloatSlider(name='epsilon', start=0., end=1., value=0.75, step=0.01, width = 250)

def get_clusters( x, y, epsilon):
    np.random.seed(123)
    db = DBSCAN(eps=epsilon, min_samples=15).fit(X)
    df['cluster'] = db.labels_
    class_plot= df.sort_values('cluster').hvplot.scatter(x, y, by='cluster', marker='o', size=100 )
    class_plot.opts(xlabel="X", ylabel="Y", width=700, height=600, tools=['hover'],  toolbar= 'left',
         title="Пространство признаков", legend_position='top_left', show_grid=True)
    return ( class_plot ) 

pn.Row(
       
    pn.pane.HoloViews(
        pn.bind(get_clusters, x, y, epsilon)
    ).servable(),
    pn.WidgetBox(
        pn.Column(
            x, y,
            "DBScan",
            epsilon,
            height = 400,
            ).servable(target='sidebar')
    ),
)

Домашнее задание.

Сформировать исходную выборку для датасета WINE используя только указанные в варианте задания классы.
Для заданных алгоритмов кластеризации и числа кластеров подобрать параметры алгоритмов обеспечивающие максимум качества кластеризации про метрике полнота. 
Сравнить результаты кластеризации различными алгоритмами между собой. Сделать вывод.

In [ ]:
import bokeh.io
bokeh.io.output_notebook()